In [1]:
#임성민 프로그래밍응용 과제
#방향키로 테트리스 블럭을 움직이고 "b" 로 빨리 내리기, 스페이스바로 한번에 내리기 입니다.
#테마곡은 게임이 종료되면 알아서 멈추고 게임 스크립트 종료시 반복설정해 놓은 부분까지만 움직입니다.
#설정량만큼 라인 클리어시 레벨이 점점 올라갑니다.
#제가 좋아하는 테트리스 모바일에서 마라톤모드를 보고 영감을 받아 계속계속 진행되는 테트리스로 만들었습니다
#제가 발견한 오류는 없으며 개인 최고기록은 25레벨 입니다!



import SimpleGUICS2Pygame.simpleguics2pygame as simplegui
import random
import time

#조작키
start_level = 0
left_move = "left"
right_move = "right"
lolling = "up"
cc_rotation = "down"
soft_drop = "b"
hard_drop = "space"



width = 10
height = 22
block_size = 20

frame_width = 16
frame_height = 22

#스코어
num_cleared = 0
score = 0
level = 0

grid = [[7 for j in range(height)] for i in range(width)]

# 블럭의 색 설정
color_Dict = {0:"Cyan", 
              1:"Orange", 
              2:"Blue", 
              3:"Purple", 
                4:"Red", 
              5:"Lime", 
              6:"Pink", 
              7:"White", 
                8: "Black"}


# 캔버스 준비시작
def draw_block(c,pos,color):
    c.draw_polygon([[pos[0],pos[1]],[pos[0]+block_size, 
                    pos[1]],[pos[0]+block_size, 
                    pos[1]+block_size],[pos[0], 
                    pos[1]+block_size]],1,"Gray",color)


#오른쪽에 글씨 써놓고 블럭그리기

def draw(c):
    global frame_height, block_size, pos_list, num_cleared, score, level, start_level
    
    c.draw_line((10*block_size,0),(10*block_size, frame_height*block_size),15, "Black")
    c.draw_text("Next Block:",(11*block_size, 1*block_size) , 12, "Black")
    c.draw_text("Lines Cleared:", (11*block_size, 6*block_size), 12, "Black")
    c.draw_text(str(num_cleared), (13*block_size, 7*block_size) , 12, "Black")
    c.draw_text("Score:", (11*block_size, 9*block_size), 12, "Black")
    c.draw_text(str(score), (13*block_size, 10*block_size), 12, "Black")
    c.draw_text("Level:", (11*block_size, 12*block_size), 12, "Black")
    c.draw_text(str(level+start_level), (13*block_size, 13*block_size) , 12, "Black")
    
#다음 피스 그리기

    global width
    next_piece_go = [(width-2)*block_size, (2)*block_size]

 #다음 피스 출발   
    for pos in pos_list.piece_dict[pos_list.next_piece]:
        draw_block(c, [pos[0]*block_size + next_piece_go[0], pos[1]*block_size +next_piece_go[1]],color_Dict[pos_list.next_piece])
    
    for i in range(width):
        for j in range(height):
            draw_block(c,[i*block_size,j*block_size], color_Dict[grid[i][j]])
            
            
class Controls:

    def __init__(self, left_move, right_movement,lolling, cc_rotation, soft_drop, hard_drop):

        self.previous_key = None
        self.left_move = left_move
        self.right_move = right_move
        self.lolling = lolling
        self.cc_rotation = cc_rotation 
        self.soft_drop = soft_drop
        self.hard_drop = hard_drop 
   
        
    def keydown_handler(self, key):
        if self.previous_key == None:
            self.previous_key = key
            self.key = key
            self.keydown() 
            self.timer = simplegui.create_timer(1000.0/15, self.keydown)
            self.timer.start()
           
                    
 #양옆으로 움직이도록 코딩    

    def keydown(self):
        global pos_list, pos
        
        lowest_val = width
        highest_val = 0
        left_blocks = []
        right_blocks = []
        for pos in pos_list.piece:
            if pos[0] < lowest_val:
                lowest_val = pos[0]
                left_blocks = []
                left_blocks.append(pos)
                
            if pos[0] == highest_val:
                left_blocks.append(pos)
                
            if pos[0] > highest_val:
                highest_val = pos[0]
                right_blocks  = []
                right_blocks .append(pos)
                
            if pos[0] == highest_val:
                right_blocks .append(pos)
                
        fall = check_fall()
        right, left = check_sideways()
        
        if self.key ==simplegui.KEY_MAP [self.left_move] and left_blocks[0][0] != 0 and left:
            
 #한줄이 채워진블럭을 하얀색으로 바꾸기

            for block in pos_list.piece:
                grid[block[0]][block[1]]= 7
            pos_list.move_piece([-1,0])
            
 #하얀색으로           
        elif self.key == simplegui.KEY_MAP [self.right_move] and right_blocks[0][0] != width-1 and right:
            for block in pos_list.piece:
                grid[block[0]][block[1]]= 7
            pos_list.move_piece([1,0])
            
            
#계속             
        elif self.key == simplegui.KEY_MAP [self.hard_drop] and fall:
            while fall:
                for block in pos_list.piece:
                    grid[block[0]][block[1]]= 7
                pos_list.move_piece([0,1])
                fall = check_fall()
                               
#계속 업데이트 해줌        
        elif self.key == simplegui.KEY_MAP [self.soft_drop] and fall:            
            for block in pos_list.piece:
                grid[block[0]][block[1]]= 7
            pos_list.move_piece([0,1])
            

        
        elif self.key == simplegui.KEY_MAP [self.lolling]:
            pos_list.rotate(True)
        
        elif self.key == simplegui.KEY_MAP[self.cc_rotation]:
            pos_list.rotate(False)
            
        
    def keyup(self, key):
        if key == self.previous_key:
            self.timer.stop()
            self.previous_key = None

class Blocks:
    
    def __init__(self):

        self.choice = random.randint(0,6) 
                               
        self.next_piece = random.randint(0,6)
        self.piece_dict = {0:[[3,0],[4,0],[5,0],[6,0]], 
                           1:[[4,1],[5,1],[6,1],[6,0]], 
                           2:[[4,0],[4,1],[5,1],[6,1]], 
                           3:[[4,1],[5,1],[6,1],[5,0]], 
                           4:[[4,0],[5,0],[5,1],[6,1]], 
                           5:[[5,0],[6,0],[4,1],[5,1]], 
                           6:[[4,0],[5,0],[4,1],[5,1]]}
        
    
        

    def create_piece(self):
        self.choice = self.next_piece
        self.next_piece = random.randint(0,6)
        self.piece = self.piece_dict[self.choice]
        
        if self.choice == 0:
            self.piece = [[3,0],[4,0],[5,0],[6,0]]
            self.rotation_offset = [[2,-1],[1,0],[0,1],[-1,2]]
            self.cc_rotation_offset = [[1,2],[0,1],[-1,0],[-2,-1]]
            
        if self.choice == 1:
            self.piece = [[4,1],[5,1],[6,1],[6,0]]
            self.rotation_offset = [[1,-1],[0,0],[-1,1],[0,2]]
            self.cc_rotation_offset =  [[1,1],[0,0],[-1,-1],[-2,0]]
            
        if self.choice == 2:
            self.piece = [[4,0],[4,1],[5,1],[6,1]]
            self.rotation_offset = [[2,0],[1,-1],[0,0],[-1,1]]
            self.cc_rotation_offset = [[0,2],[1,1],[0,0],[-1,-1]]
            
        if self.choice == 3:
            self.piece = [[4,1],[5,1],[6,1],[5,0]]
            self.rotation_offset = [[1,-1],[0,0],[-1,1],[1,1]]
            self.cc_rotation_offset = [[1,1],[0,0],[-1,-1],[-1,1]]
            
        if self.choice == 4:
            self.piece = [[4,0],[5,0],[5,1],[6,1]]
            self.rotation_offset = [[2,0],[1,1],[0,0],[-1,1]]
            self.cc_rotation_offset =  [[0,2],[-1,1],[0,0],[-1,-1]]
            
        if self.choice == 5:
            self.piece = [[5,0],[6,0],[4,1],[5,1]]
            self.rotation_offset = [[1,1],[0,2],[1,-1],[0,0]]
            self.cc_rotation_offset =  [[-1,1],[-2,0],[1,1],[0,0]]
            
        if self.choice == 6:
            self.piece = [[4,0],[5,0],[4,1],[5,1]]
            self.rotation_offset = [[0,0],[0,0],[0,0],[0,0]]
            self.cc_rotation_offset = [[0,0],[0,0],[0,0],[0,0]]
            
        #블럭을 처음부터 그리기 시작
        self.move_piece([0,0]) 
            
            
#블록 업데이트 예시 [0,1]            

    def move_piece(self, offset):
        move = self.piece
        
        for block_idx in range(len(move)):
            self.piece[block_idx] = [move[block_idx][0]+ offset[0], 
                                    move[block_idx][1]+ offset[1]]
        for block in self.piece: 
            grid[block[0]][block[1]] = self.choice
            
            
#잘돌아 가는가, CW = 시계방향이면 True, 다른건 다 False            
    def rotate(self, CW):       
        global grid
        
        if CW == True:
            rot_offset = self.rotation_offset
        else: 
            rot_offset = self.cc_rotation_offset
        if check_rotation(CW):
        
            for block in self.piece:
                grid[block[0]][block[1]]= 7
                
         
            move_piece = self.piece
            
            for pos_idx in range(len(move_piece)):
                self.piece[pos_idx][0] =self.piece[pos_idx] [0] + rot_offset[pos_idx][0]
                self.piece[pos_idx][1] =self.piece[pos_idx] [1] + rot_offset[pos_idx][1]
                
        
            if CW:
                for i in [self.rotation_offset, self.cc_rotation_offset]:
                    rot_move = i
                    for rot_idx in range(len(rot_move)):
                        i[rot_idx][1] = i[rot_idx][1] * (-1)
                        i[rot_idx].reverse()
            else:
                for i in [self.rotation_offset, self.cc_rotation_offset]:
                    rot_move = i
                    for rot_idx in range(len(rot_move)):
                        i[rot_idx][0] = i[rot_idx][0] * (-1)
                        i[rot_idx].reverse()
                
    
            for pos in self.piece:
                grid[pos[0]][pos[1]] = self.choice
                


                
#블럭이 잘돌아 가는가, CW = 시계방향이면 True, 다른건 다 False
def check_rotation(CW):
   
    global pos_list, grid
    if CW == True:
        rot_offset = pos_list.rotation_offset
    else: 
        rot_offset = pos_list.cc_rotation_offset
        
    rotation = True
    check_pos = []
    
    for pos_idx in range(len(pos_list.piece)):
        check_pos.append([pos_list.piece[pos_idx][0]+ rot_offset[pos_idx][0], pos_list.piece[pos_idx][1]+ rot_offset[pos_idx][1]])
        
    for i in check_pos:
        if i[0] < 0 or i[1] < 0 or i[0] > width-1 or i[1] > height-1:
            rotation = False
            return rotation
        
    for col_idx in range(len(grid)):
        for row_idx in range(len(grid[0])):
            if grid[col_idx][row_idx] != 7 and [col_idx,row_idx] in check_pos and [col_idx,row_idx] not in pos_list.piece:
                rotation = False
                return rotation
      
    return rotation
     
    
#블럭 잘 내려오는지 확인    
def check_fall():
    
    fall = True
    global pos_list
    
    for pos in pos_list.piece:
        if pos[1] == height-1: 
            fall = False
            break
            
        elif grid[pos[0]][pos[1]+1]!= 7 and ([pos[0], pos[1]+1] not in pos_list.piece):
            fall = False
            break
            
    return fall

def check_sideways():
    global pos_list
    
    right = True
    left = True
    
    for pos in pos_list.piece:
        if pos[0] == width -1:
            right = False
        elif grid[pos[0]+1][pos[1]]!= 7 and ([pos[0]+1,pos[1]] not in pos_list.piece):
            right = False
        if grid[pos[0]-1][pos[1]]!= 7 and ([pos[0]-1,pos[1]] not in pos_list.piece):
            
            left = False
    return right, left

#테트리스 배경음악을 깔아줍시다
class Theme:

    def __init__(self):
        self.sound = simplegui.load_sound("file:///C:/Users/89624/Desktop/20182890_py/20182890.ogg")
        self.sound.play()
        self.sound.set_volume(1)
        self.last_time = time.time()

    def check_restart(self):
        if time.time() - self.last_time > 48: 
            self.sound.rewind()
            self.sound.play()
            self.last_time = time.time()

    def stop(self):
        self.sound.pause() 


        
#블록위치 업데이트
def my_update():
    global pos_list, grid, num_cleared, song_them, score, level, start_level
    
    lines_bonus = {0:0, 1:40, 2:100, 3:300, 4:1200}

    song_theme.check_restart()
    
    
#가장 아래있는 블록을 확인한다
#0이 제일 위이고 21이 가장 아래로 21줄을 가득 채우면 게임종료

    lowest_val = -1
    lowest_blocks = []
    
    for pos in pos_list.piece:
        if pos[1] > lowest_val:
            lowest_val = pos[1]
            lowest_blocks = []
            lowest_blocks.append(pos)
        if pos[1] == lowest_val:
            lowest_blocks.append(pos)
            
    fall = check_fall()

    #블럭움직이기
    if fall == True:
        
        #오래된 블럭을 하얗게
        for block in pos_list.piece:
            grid[block[0]][block[1]]= 7
            
        #새로운 블럭을 그리기
        pos_list.move_piece([0,1])
    
    lose_flag = False
    
    if fall == False:
        for pos in pos_list.piece:
            if pos[1] == 0:
                lose_flag = True
                break
    
    if lose_flag == False and fall == False:
        remove_lines = check_line()
        num_cleared += len(remove_lines)
        score += lines_bonus[len(remove_lines)]* (start_level+level+1)
        
        if num_cleared >= (level+1)*10:
            level += 1
        clear_lines(remove_lines)
        pos_list.create_piece()
        
    if lose_flag == True:
        print ("YOU are LEVEL : " + str(level))
        t.stop()
        grid = [[8 for j in range(height)] for i in range(width)]

        song_theme.stop()
    
#행이 잘 사라지는지        
def check_line():    
    global grid

    remove_lines = []
    for i in range(height):
        remove_lines.append(i)
        
    move = grid
    for row_idx in range(len(move[0])):
        for col_idx in range(len(move)):
            if move[col_idx][row_idx] == 7:
                if row_idx in remove_lines:
                    remove_lines.remove(row_idx)

    return remove_lines

#선 지우기
def clear_lines(remove_lines):
    global grid
    
    move = grid
    
    for line_num in remove_lines:
        for col_idx in range(len(move)):
            grid[col_idx][line_num] = 7

#꽉찬 줄을 지우고 새로 업데이트
    move = grid
    
    for line in remove_lines:
        for row_idx in range(line + 1):
            for col_idx in range(len(move)):
                if line - row_idx != 0:
                    grid[col_idx][line - row_idx] = move[col_idx][line - row_idx-1]
                    
                    
                    

#첫번째 블록 위치 
pos_list = Blocks()
pos_list.create_piece()
song_theme = Theme()
controls = Controls(left_move, right_move, lolling, cc_rotation,  soft_drop, hard_drop)

frame = simplegui.create_frame("Tetris", frame_width * block_size, frame_height*block_size, 200)
frame.set_canvas_background("White")
frame.set_draw_handler(draw)
frame.set_keydown_handler(controls.keydown_handler)
frame.set_keyup_handler(controls.keyup) 


t = simplegui.create_timer(1000.0/(start_level+level+1), my_update)


t.start()

frame.start()


pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html
